# Quick start

In [1]:
import numpy as np
import networkx as nx
import math
import time
import pickle as pkl
import scipy
import sys
sys.path.insert(0, '/home/azzolin/AHK')
import utils

from ahk import AHK_graphon
from ahk_generators import data_colors, ahk_sbm, sample_data

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

/home/azzolin/miniconda3/envs/digress2_clone/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Define an AHK model representing a stochastic block model:

colprobs = np.array([0.2, 0.3, 0.5]) #np.array([1])
edgeprobs = np.array([[0.6,0.1,0.1],[0.1,0.9,0.1],[0.1,0.1,0.8]])

# sbm = ahk_sbm(colprobs, edgeprobs, False)

In [27]:
### Sample random graphs from the model

# minnodes = 8
# maxnodes = 30
# samplesize = 5

# sample = sample_data(sbm,samplesize,minnodes,maxnodes)

### these sampled graphs are now instances of utils.World
### They are converted to networkx graphs for plotting

#
#for g in sample:
#    gnx=g.to_nx()
#    nodecols=(list(gnx.nodes()[j]['features'] for j in gnx.nodes()))
#    nx.draw_networkx(gnx,node_color=nodecols,cmap='Set3')
#    plt.show()


In [36]:
# g = sample[0]
# g = g.to_nx()
# g.nodes(data=True)
# g.edges(data=True)
# g = nx.Graph(g)
# g.edges()[(0,2)]["bici"] = 2
# g.edges(data=True)

In [39]:
# for g in range(5):
#     g = nx.stochastic_block_model([5, 5, 5], edgeprobs)
#     nx.draw_networkx(g, cmap='Set3')
#     plt.show()

## Learning

In [16]:
NUM_GRAPHS = 100

traindata = []
for _ in range(NUM_GRAPHS):
    g = nx.stochastic_block_model([5,5,5], edgeprobs)
    # nx.set_node_attributes(g, 0, "features") # DO NOT ADD NODE ATTRIBS
    g = utils.nx_to_world(g)
    traindata.append(g)
g.sig.unaries


# Initializing a model. This requires to define the histogram partitioning of the [0,1] interval
# that the model uses
# The first argument in the constructor is the signature of the graphs we are dealing with, i.e.
# a specification of the attributes and relations


binbounds = utils.uni_bins(3)
model_learned = AHK_graphon(traindata[0].sig, binbounds)

# Defining settings:
settings={}
settings['num_pi_b']=50
settings['batchsize']=10
settings['soft']=0.01
settings['numepochs']=30
settings['learn_bins']=False
settings['early_stop']=5
settings['with_trace']=False

#Adam params:
settings['ad_alpha']=0.01
settings['ad_beta1']=0.99
settings['ad_beta2']=0.9
settings['ad_epsilon']=10e-8

settings['method']="adam"

In [17]:
# Random initialization and learning
model_learned.rand_init()
best, loglik, trace = model_learned.learn(settings, traindata)

  0%|          | 0/10 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
sample = sample_data(model_learned, 5, 5, 30)

for g in sample:
   gnx = g.to_nx()
   nodecols = (list(gnx.nodes()[j]['features'] for j in gnx.nodes()))
   nx.draw_networkx(gnx, node_color=nodecols, cmap='Set3')
   plt.show()